# Fun with funnels

### Установка 

В первую очередь нам понадобится ClickHouse.
ClickHouse можно установить на Linux и MAC (подробно про установку написано в документации https://clickhouse.yandex/).

Также удобно запустить CH в Docker'e: первая команда поднимает clickhouse-server на порту 8123, а вторая - позволяет подключиться к консольному ClickHouse клиенту.
```
docker run -d --name clickhouse-server --publish=8123:8123 --publish=9000:9000 yandex/clickhouse-server
docker run -it --rm --link clickhouse-server:9000 yandex/clickhouse-client --host clickhouse-server
```

Из библиотек для python нам понадобятся:
* `requests`
* `plotly`
* `pandas`

Данные можно скачать тут - https://yadi.sk/d/xv6iNs5Tkh1G8A

###  Функции для интеграции с ClickHouse

Напишем функции для интеграции с ClickHouse: первая функция просто возвращает результат из DataBase, вторая же преобразует его в pandas DataFrame.

Также напишем сразу удобную функцию для загрузки данных.

In [6]:
HOST = 'http://localhost:8123'
import requests
import pandas as pd
import io 
pd.set_option('display.max_colwidth', 1000)

import sys
stdin, stdout, stderr = sys.stdin, sys.stdout, sys.stderr
reload(sys)
sys.stdin, sys.stdout, sys.stderr = stdin, stdout, stderr
sys.setdefaultencoding('utf8')

NameError: name 'reload' is not defined

In [5]:

def get_clickhouse_data(query, host = HOST, connection_timeout = 1500):
    r = requests.post(host, params = {'query': query}, timeout = connection_timeout)
    if r.status_code == 200:
        return r.text
    else:
        raise ValueError, r.text
        
def get_clickhouse_df(query, host = HOST, connection_timeout = 1500):
    data = get_clickhouse_data(query, host, connection_timeout) 
    df = pd.read_csv(StringIO.StringIO(data), sep = '\t')
    return df

def upload(table, content, host=HOST):
    content = content.encode('utf-8')
    query_dict = {
             'query': 'INSERT INTO ' + table + ' FORMAT TabSeparatedWithNames '
        }
    r = requests.post(host, data=content, params=query_dict)
    result = r.text
    if r.status_code == 200:
        return result
    else:
        raise ValueError(r.text)

SyntaxError: invalid syntax (<ipython-input-5-37bb0877108c>, line 7)

### Загружаем данные

In [2]:
df = pd.read_csv('web_data.csv', sep = '\t', dtype = str)
df.head()

NameError: name 'pd' is not defined

In [3]:
df.shape

(6369907, 7)

In [6]:
q = 'drop table if exists visits'
get_clickhouse_data(q)

q = '''
create table visits (
    ClientID UInt64,
    EventTime DateTime,
    EventDate Date,
    Browser String,
    DeviceCategory String,
    TraficSource String,
    URL String
) ENGINE = MergeTree(EventDate, intHash32(ClientID), (EventDate, intHash32(ClientID)), 8192)
'''

get_clickhouse_data(q)

u''

In [7]:
%%time
upload(
    'visits',
    df[['ClientID', 'EventTime', 'EventDate', 'Browser', 'DeviceCategory', 'TraficSource', 'URL']]\
    .to_csv(index = False, sep = '\t'))

CPU times: user 29.4 s, sys: 6.2 s, total: 35.6 s
Wall time: 52.3 s


u''

### Разминка: простые запросы

In [11]:
q = '''
SELECT
    URL,
    uniq(ClientID) as users,
    count() as hits
FROM visits
GROUP BY URL
ORDER BY users DESC
FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,URL,users,hits
0,supermarket.ru/product,723755,2261197
1,supermarket.ru/catalog,537645,1964040
2,supermarket.ru/main,455987,866494
3,supermarket.ru/promo,175554,326333
4,supermarket.ru/search,161922,412294
5,supermarket.ru/promo_action_1,45337,65111
6,supermarket.ru/shops,45005,81199
7,supermarket.ru/blog,34857,49515
8,supermarket.ru/cards,25138,94732
9,supermarket.ru/basket,23118,86181


In [12]:
q = '''
SELECT
    ClientID,
    groupArray(URL) as path,
    arraySlice(groupArray(URL), 1, 5) as path_lim
FROM
    (SELECT * FROM visits ORDER BY ClientID, EventTime)
GROUP BY ClientID
LIMIT 10
FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,ClientID,path,path_lim
0,1464279176664798753,"['supermarket.ru/main','supermarket.ru/catalog','supermarket.ru/product','supermarket.ru/finish_reserve','supermarket.ru/catalog','supermarket.ru/product','supermarket.ru/product','supermarket.ru/product']","['supermarket.ru/main','supermarket.ru/catalog','supermarket.ru/product','supermarket.ru/finish_reserve','supermarket.ru/catalog']"
1,146446839869868089,"['supermarket.ru/catalog','supermarket.ru/catalog','supermarket.ru/product','supermarket.ru/catalog']","['supermarket.ru/catalog','supermarket.ru/catalog','supermarket.ru/product','supermarket.ru/catalog']"
2,1469184715541189697,"['supermarket.ru/main','supermarket.ru/main','supermarket.ru/promo_action_1','supermarket.ru/main','supermarket.ru/search','supermarket.ru/product','supermarket.ru/product']","['supermarket.ru/main','supermarket.ru/main','supermarket.ru/promo_action_1','supermarket.ru/main','supermarket.ru/search']"
3,1468981110571788812,['supermarket.ru/catalog'],['supermarket.ru/catalog']
4,1469025223567819473,"['supermarket.ru/main','supermarket.ru/search','supermarket.ru/product','supermarket.ru/search','supermarket.ru/product','supermarket.ru/product','supermarket.ru/product','supermarket.ru/product','supermarket.ru/product','supermarket.ru/product','supermarket.ru/promo','supermarket.ru/search','supermarket.ru/product','supermarket.ru/catalog','supermarket.ru/catalog','supermarket.ru/product']","['supermarket.ru/main','supermarket.ru/search','supermarket.ru/product','supermarket.ru/search','supermarket.ru/product']"
5,1468853245481263835,"['supermarket.ru/catalog','supermarket.ru/product','supermarket.ru/catalog','supermarket.ru/product']","['supermarket.ru/catalog','supermarket.ru/product','supermarket.ru/catalog','supermarket.ru/product']"
6,14677017851011543978,['supermarket.ru/product'],['supermarket.ru/product']
7,1461304408893575765,"['supermarket.ru/main','supermarket.ru/catalog','supermarket.ru/product','supermarket.ru/catalog','supermarket.ru/product','supermarket.ru/catalog','supermarket.ru/catalog','supermarket.ru/catalog']","['supermarket.ru/main','supermarket.ru/catalog','supermarket.ru/product','supermarket.ru/catalog','supermarket.ru/product']"
8,1469010509738994548,"['supermarket.ru/main','supermarket.ru/catalog','supermarket.ru/catalog','supermarket.ru/catalog','supermarket.ru/product','supermarket.ru/product']","['supermarket.ru/main','supermarket.ru/catalog','supermarket.ru/catalog','supermarket.ru/catalog','supermarket.ru/product']"
9,1469770190831702596,"['supermarket.ru/main','supermarket.ru/catalog','supermarket.ru/catalog','supermarket.ru/product','supermarket.ru/catalog','supermarket.ru/product','supermarket.ru/catalog','supermarket.ru/product','supermarket.ru/product','supermarket.ru/product','supermarket.ru/product','supermarket.ru/catalog','supermarket.ru/product','supermarket.ru/catalog','supermarket.ru/catalog','supermarket.ru/product','supermarket.ru/catalog','supermarket.ru/product','supermarket.ru/catalog','supermarket.ru/product','supermarket.ru/product','supermarket.ru/catalog']","['supermarket.ru/main','supermarket.ru/catalog','supermarket.ru/catalog','supermarket.ru/product','supermarket.ru/catalog']"


In [13]:
q = '''
SELECT
    path_lim,
    count() as users
FROM
    (SELECT
        ClientID,
        arraySlice(groupArray(URL), 1, 5) as path_lim
    FROM
        (SELECT * FROM visits ORDER BY ClientID, EventTime)
    GROUP BY ClientID)
GROUP BY path_lim
ORDER BY users desc
LIMIT 10
FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,path_lim,users
0,['supermarket.ru/product'],239251
1,['supermarket.ru/catalog'],89892
2,['supermarket.ru/promo'],49317
3,"['supermarket.ru/product','supermarket.ru/product']",47269
4,['supermarket.ru/main'],30156
5,"['supermarket.ru/main','supermarket.ru/catalog']",20251
6,"['supermarket.ru/catalog','supermarket.ru/catalog']",19953
7,"['supermarket.ru/product','supermarket.ru/product','supermarket.ru/product']",15781
8,"['supermarket.ru/main','supermarket.ru/search']",14456
9,"['supermarket.ru/main','supermarket.ru/catalog','supermarket.ru/product','supermarket.ru/catalog','supermarket.ru/product']",13691


## Расчет funnels

#### Подход в лоб

In [14]:
q = '''
SELECT
    ClientID,
    max(URL = 'supermarket.ru/main') as step1_achieved,
    sequenceMatch('(?1).*(?2)')(EventTime, URL = 'supermarket.ru/main', 
      URL = 'supermarket.ru/catalog') as step2_achieved,
    sequenceMatch('(?1).*(?2).*(?3)')(EventTime, URL = 'supermarket.ru/main', 
      URL = 'supermarket.ru/catalog', URL = 'supermarket.ru/product') as step3_achieved
FROM visits
GROUP BY ClientID
LIMIT 10
FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,ClientID,step1_achieved,step2_achieved,step3_achieved
0,1469465276765527839,0,0,0
1,146830716081902312,0,0,0
2,146902420610673921,0,0,0
3,1469978444925864701,0,0,0
4,1469193894524341385,1,0,0
5,146784048029663783,0,0,0
6,1468438934442042316,1,1,0
7,1469191895881108706,0,0,0
8,1469346109660689512,0,0,0
9,1460911046620105027,0,0,0


In [15]:
q = '''
SELECT
    sum(step1_achieved) as step1,
    sum(step2_achieved) as step2,
    sum(step3_achieved) as step3
    --round(100.*step2/step1, 2) as step2_to_1,
    --round(100.*step3/step1, 2) as step3_to_1
FROM
    (SELECT
        ClientID,
        max(URL = 'supermarket.ru/main') as step1_achieved,
        sequenceMatch('(?1).*(?2)')(EventTime, URL = 'supermarket.ru/main', 
          URL = 'supermarket.ru/catalog') as step2_achieved,
        sequenceMatch('(?1).*(?2).*(?3)')(EventTime, URL = 'supermarket.ru/main', 
          URL = 'supermarket.ru/catalog', URL = 'supermarket.ru/product') as step3_achieved
    FROM visits
    GROUP BY ClientID)
FORMAT TabSeparatedWithNames
'''

fdf = get_clickhouse_df(q)

In [16]:
fdf

,step1,step2,step3
0,452714,285886,193191


Визуализируем

In [17]:
from plotly.offline import init_notebook_mode, iplot
import plotly
import plotly.graph_objs as go

init_notebook_mode(connected=True)

In [18]:
colors = colors = ['#d54936', '#faca34', '#437cba', '#8bc34a', '#795548', '#309688', '#000000', '#40bcd4', '#9e9e9e', '#3ca9f4']
phases = ['Step %d' % i for i in range(1, 4)]
values = fdf.iloc[0].values

In [19]:
def plot_funnel(phases, values):
    n_phase = len(phases)
    plot_width = 500.

    # height of a section and difference between sections 
    section_h = 100
    section_d = 10

    # multiplication factor to calculate the width of other sections
    unit_width = plot_width / max(values)

    # width of each funnel section relative to the plot width
    phase_w = [int(value * unit_width) for value in values]
    print phase_w

    # plot height based on the number of sections and the gap in between them
    height = section_h * n_phase + section_d * (n_phase - 1)
    
    # list containing all the plot shapes
    shapes = []

    # list containing the Y-axis location for each section's name and value text
    label_y = []

    for i in range(n_phase):
            if (i == n_phase-1):
                    points = [phase_w[i] / 2, height, phase_w[i] / 2, height - section_h]
            else:
                    points = [phase_w[i] / 2, height, phase_w[i+1] / 2, height - section_h]

            path = 'M {0} {1} L {2} {3} L -{2} {3} L -{0} {1} Z'.format(*points)

            shape = {
                    'type': 'path',
                    'path': path,
                    'fillcolor': colors[i],
                    'line': {
                        'width': 1,
                        'color': colors[i]
                    }
            }
            shapes.append(shape)

            # Y-axis location for this section's details (text)
            label_y.append(height - (section_h / 2))

            height = height - (section_h + section_d)

    # For phase names
    label_trace = go.Scatter(
        x=[-350]*n_phase,
        y=label_y,
        mode='text',
        text=phases,
        textfont=dict(
            color='rgb(40,40,40)',
            size=15
        )
    )

    # For phase values
    value_trace = go.Scatter(
        x=[350]*n_phase,
        y=label_y,
        mode='text',
        text=values,
        textfont=dict(
            color='rgb(40,40,40)',
            size=15
        )
    )

    data = [label_trace, value_trace]

    layout = go.Layout(
        title="<b>Funnel Chart</b>",
        titlefont=dict(
            size=20,
            color='rgb(0,0,0)'
        ),
        shapes=shapes,
        height=560,
        width=800,
        showlegend=False,
        paper_bgcolor='rgba(255,255,255,1)',
        plot_bgcolor='rgba(255,255,255,1)',
        xaxis=dict(
            showticklabels=False,
            zeroline=False,
            showgrid=False,
            range=[-450, 450]
        ),
        yaxis=dict(
            showticklabels=False,
            zeroline=False,
            showgrid=False
        )
    )

    fig = go.Figure(data=data, layout=layout)
    iplot(fig, show_link=False)

In [20]:
plot_funnel(phases, values)

[500, 315, 213]


### Добавим еще один шаг

In [21]:
q = '''
SELECT
    sum(step1_achieved) as step1,
    sum(step2_achieved) as step2,
    sum(step3_achieved) as step3,
    sum(step4_achieved) as step4
FROM
    (SELECT
        ClientID,
        max(URL = 'supermarket.ru/main') as step1_achieved,
        sequenceMatch('(?1).*(?2)')(EventTime, URL = 'supermarket.ru/main', 
          URL = 'supermarket.ru/catalog') as step2_achieved,
        sequenceMatch('(?1).*(?2).*(?3)')(EventTime, URL = 'supermarket.ru/main', 
          URL = 'supermarket.ru/catalog', URL = 'supermarket.ru/product') as step3_achieved,
        sequenceMatch('(?1).*(?2).*(?3).*(?4)')(EventTime, URL = 'supermarket.ru/main', 
          URL = 'supermarket.ru/catalog', URL = 'supermarket.ru/product', 
          URL = 'supermarket.ru/basket') as step4_achieved
    FROM visits
    GROUP BY ClientID)
FORMAT TabSeparatedWithNames
'''

fdf = get_clickhouse_df(q)

ValueError: Code: 160, e.displayText() = DB::Exception: Pattern application proves too difficult, exceeding max iterations (1000000), e.what() = DB::Exception


### Упростим обработку событий

In [22]:
q = '''
SELECT
    ClientID,
    groupArray(EventTime) as times,
    groupArray(URL) as urls,
    arrayEnumerate(urls) as index
FROM
    (SELECT * FROM visits ORDER BY ClientID, EventTime)
GROUP BY ClientID
LIMIT 3
FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,ClientID,times,urls,index
0,1464279176664798753,"['2016-07-04 16:53:54','2016-07-04 16:54:33','2016-07-04 16:55:36','2016-07-04 16:57:05','2016-07-04 16:57:23','2016-07-04 17:46:06','2016-07-05 19:38:31','2016-07-05 19:38:53']","['supermarket.ru/main','supermarket.ru/catalog','supermarket.ru/product','supermarket.ru/finish_reserve','supermarket.ru/catalog','supermarket.ru/product','supermarket.ru/product','supermarket.ru/product']","[1,2,3,4,5,6,7,8]"
1,146446839869868089,"['2016-07-05 16:31:33','2016-07-05 16:34:07','2016-07-05 16:34:39','2016-07-05 16:35:37']","['supermarket.ru/catalog','supermarket.ru/catalog','supermarket.ru/product','supermarket.ru/catalog']","[1,2,3,4]"
2,1469184715541189697,"['2016-07-22 13:51:59','2016-07-22 13:52:58','2016-07-22 13:53:41','2016-07-22 13:54:14','2016-07-22 13:55:11','2016-07-22 13:55:45','2016-07-22 13:57:30']","['supermarket.ru/main','supermarket.ru/main','supermarket.ru/promo_action_1','supermarket.ru/main','supermarket.ru/search','supermarket.ru/product','supermarket.ru/product']","[1,2,3,4,5,6,7]"


In [23]:
q = '''
SELECT
    ClientID,
    groupArray(EventTime) as times,
    groupArray(URL) as urls,
    arrayEnumerate(urls) as indexes,
    arrayFilter(url, index -> (index = 1) or (urls[index] != urls[index - 1]), urls, indexes) as filt_urls,
    arrayFilter(time, index -> (index = 1) or (urls[index] != urls[index - 1]), times, indexes) as filt_times
FROM
    (SELECT * FROM visits SAMPLE 1/5 ORDER BY ClientID, EventTime)
GROUP BY ClientID
LIMIT 3
FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,ClientID,times,urls,indexes,filt_urls,filt_times
0,1468821574632687631,"['2016-07-18 08:48:01','2016-07-18 08:48:40','2016-07-18 08:48:57','2016-07-18 08:51:36','2016-07-18 08:52:03','2016-07-18 08:55:11','2016-07-18 08:55:34']","['supermarket.ru/catalog','supermarket.ru/catalog','supermarket.ru/catalog','supermarket.ru/catalog','supermarket.ru/catalog','supermarket.ru/catalog','supermarket.ru/catalog']","[1,2,3,4,5,6,7]",['supermarket.ru/catalog'],['2016-07-18 08:48:01']
1,1468839784453828719,"['2016-07-18 14:03:06','2016-07-18 14:21:14','2016-07-18 16:31:26','2016-07-18 16:32:24']","['supermarket.ru/main','supermarket.ru/main','supermarket.ru/main','supermarket.ru/catalog']","[1,2,3,4]","['supermarket.ru/main','supermarket.ru/catalog']","['2016-07-18 14:03:06','2016-07-18 16:32:24']"
2,1470060064805210536,"['2016-08-01 17:01:03','2016-08-01 17:01:17','2016-08-01 17:01:27']","['supermarket.ru/main','supermarket.ru/search','supermarket.ru/product']","[1,2,3]","['supermarket.ru/main','supermarket.ru/search','supermarket.ru/product']","['2016-08-01 17:01:03','2016-08-01 17:01:17','2016-08-01 17:01:27']"


In [24]:
q = '''
SELECT
    ClientID,
    URL,
    EventTime
FROM
    (SELECT
        ClientID,
        groupArray(EventTime) as times,
        groupArray(URL) as urls,
        arrayEnumerate(urls) as indexes,
        arrayFilter(url, index -> (index = 1) or (urls[index] != urls[index - 1]), urls, indexes) as filt_urls,
        arrayFilter(time, index -> (index = 1) or (urls[index] != urls[index - 1]), times, indexes) as filt_times
    FROM
        (SELECT * FROM visits SAMPLE 1/5 ORDER BY ClientID, EventTime)
    GROUP BY ClientID
    LIMIT 3)
ARRAY JOIN filt_urls as URL, filt_times as EventTime
FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,ClientID,URL,EventTime
0,1468821574632687631,supermarket.ru/catalog,2016-07-18 08:48:01
1,1468839784453828719,supermarket.ru/main,2016-07-18 14:03:06
2,1468839784453828719,supermarket.ru/catalog,2016-07-18 16:32:24
3,1470060064805210536,supermarket.ru/main,2016-08-01 17:01:03
4,1470060064805210536,supermarket.ru/search,2016-08-01 17:01:17
5,1470060064805210536,supermarket.ru/product,2016-08-01 17:01:27


In [25]:
%%time
q = '''
SELECT
    5*sum(step1_achieved) as step1,
    5*sum(step2_achieved) as step2,
    5*sum(step3_achieved) as step3,
    5*sum(step4_achieved) as step4
FROM
    (SELECT
        ClientID,
        max(URL = 'supermarket.ru/main') as step1_achieved,
        sequenceMatch('(?1).*(?2)')(EventTime, URL = 'supermarket.ru/main', 
          URL = 'supermarket.ru/catalog') as step2_achieved,
        sequenceMatch('(?1).*(?2).*(?3)')(EventTime, URL = 'supermarket.ru/main', 
          URL = 'supermarket.ru/catalog', URL = 'supermarket.ru/product') as step3_achieved,
        sequenceMatch('(?1).*(?2).*(?3).*(?4)')(EventTime, URL = 'supermarket.ru/main', 
          URL = 'supermarket.ru/catalog', URL = 'supermarket.ru/product', 
          URL = 'supermarket.ru/basket') as step4_achieved
    FROM
        (SELECT
            ClientID,
            groupArray(EventTime) as times,
            groupArray(URL) as urls,
            arrayEnumerate(urls) as indexes,
            arrayFilter(url, index -> (index = 1) or (urls[index] != urls[index - 1]), urls, indexes) as filt_urls,
            arrayFilter(time, index -> (index = 1) or (urls[index] != urls[index - 1]), times, indexes) as filt_times
        FROM
            (SELECT * FROM visits SAMPLE 1/5 ORDER BY ClientID, EventTime)
        GROUP BY ClientID)
    ARRAY JOIN filt_urls as URL, filt_times as EventTime
    GROUP BY ClientID)
FORMAT TabSeparatedWithNames
'''

fdf = get_clickhouse_df(q)

CPU times: user 6.4 ms, sys: 2.37 ms, total: 8.77 ms
Wall time: 3.9 s


In [26]:
phases = ['Step %d' % i for i in range(1, 5)]
values = fdf.iloc[0].values

plot_funnel(phases, values)

[500, 315, 212, 10]


### Добавим еще шагов

In [27]:
%%time
q = '''
SELECT
    5*sum(step1_achieved) as step1,
    5*sum(step2_achieved) as step2,
    5*sum(step3_achieved) as step3,
    5*sum(step4_achieved) as step4,
    5*sum(step5_achieved) as step5,
    5*sum(step6_achieved) as step6
FROM
    (SELECT
        ClientID,
        max(URL = 'supermarket.ru/main') as step1_achieved,
        sequenceMatch('(?1).*(?2)')(EventTime, URL = 'supermarket.ru/main', 
          URL = 'supermarket.ru/catalog') as step2_achieved,
        sequenceMatch('(?1).*(?2).*(?3)')(EventTime, URL = 'supermarket.ru/main', 
          URL = 'supermarket.ru/catalog', URL = 'supermarket.ru/product') as step3_achieved,
        sequenceMatch('(?1).*(?2).*(?3).*(?4)')(EventTime, URL = 'supermarket.ru/main', 
          URL = 'supermarket.ru/catalog', URL = 'supermarket.ru/product', 
          URL = 'supermarket.ru/basket') as step4_achieved,
        sequenceMatch('(?1).*(?2).*(?3).*(?4).*(?5)')(EventTime, URL = 'supermarket.ru/main', 
          URL = 'supermarket.ru/catalog', URL = 'supermarket.ru/product', 
          URL = 'supermarket.ru/basket', URL = 'supermarket.ru/finish_order') as step5_achieved,
        sequenceMatch('(?1).*(?2).*(?3).*(?4).*(?5).*(?6)')(EventTime, URL = 'supermarket.ru/main', 
          URL = 'supermarket.ru/catalog', URL = 'supermarket.ru/product', 
          URL = 'supermarket.ru/basket', URL = 'supermarket.ru/finish_order', URL = 'supermarket.ru/user_profile') as step6_achieved
    FROM
        (SELECT
            ClientID,
            groupArray(EventTime) as times,
            groupArray(URL) as urls,
            arrayEnumerate(urls) as indexes,
            arrayFilter(url, index -> (index = 1) or (urls[index] != urls[index - 1]), urls, indexes) as filt_urls,
            arrayFilter(time, index -> (index = 1) or (urls[index] != urls[index - 1]), times, indexes) as filt_times
        FROM
            (SELECT * FROM visits SAMPLE 1/5 ORDER BY ClientID, EventTime)
        GROUP BY ClientID)
    ARRAY JOIN filt_urls as URL, filt_times as EventTime
    GROUP BY ClientID)
FORMAT TabSeparatedWithNames
'''

fdf = get_clickhouse_df(q)

ValueError: Code: 160, e.displayText() = DB::Exception: Pattern application proves too difficult, exceeding max iterations (1000000), e.what() = DB::Exception


## Пойдем другим путем

In [28]:
q = '''
SELECT
    ClientID,
    groupArray(cast(EventTime as UInt64)) as times,
    groupArray(URL) as urls,
    arrayFilter(time, url -> url = 'supermarket.ru/main', times, urls)[1] as step1_time
FROM
    (SELECT * FROM visits ORDER BY ClientID, EventTime)
GROUP BY ClientID
LIMIT 5
FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,ClientID,times,urls,step1_time
0,1464279176664798753,"[1467651234,1467651273,1467651336,1467651425,1467651443,1467654366,1467747511,1467747533]","['supermarket.ru/main','supermarket.ru/catalog','supermarket.ru/product','supermarket.ru/finish_reserve','supermarket.ru/catalog','supermarket.ru/product','supermarket.ru/product','supermarket.ru/product']",1467651234
1,146446839869868089,"[1467736293,1467736447,1467736479,1467736537]","['supermarket.ru/catalog','supermarket.ru/catalog','supermarket.ru/product','supermarket.ru/catalog']",0
2,1469184715541189697,"[1469195519,1469195578,1469195621,1469195654,1469195711,1469195745,1469195850]","['supermarket.ru/main','supermarket.ru/main','supermarket.ru/promo_action_1','supermarket.ru/main','supermarket.ru/search','supermarket.ru/product','supermarket.ru/product']",1469195519
3,1468981110571788812,[1468995858],['supermarket.ru/catalog'],0
4,1469025223567819473,"[1469036023,1469036035,1469036063,1469036094,1469036120,1469036136,1469036148,1469036190,1469036515,1469036660,1469036798,1469036815,1469036827,1469037142,1469037147,1469037168]","['supermarket.ru/main','supermarket.ru/search','supermarket.ru/product','supermarket.ru/search','supermarket.ru/product','supermarket.ru/product','supermarket.ru/product','supermarket.ru/product','supermarket.ru/product','supermarket.ru/product','supermarket.ru/promo','supermarket.ru/search','supermarket.ru/product','supermarket.ru/catalog','supermarket.ru/catalog','supermarket.ru/product']",1469036023


In [29]:
q = '''
SELECT
    ClientID,
    groupArray(cast(EventTime as UInt64)) as times,
    groupArray(URL) as urls,
    arrayFilter(time, url -> url = 'supermarket.ru/main', times, urls)[1] as step1_time,
    arrayFilter(time, url -> url = 'supermarket.ru/catalog' and step1_time != 0 and time >= step1_time, times, urls)[1] as step2_time
FROM
    (SELECT * FROM visits ORDER BY ClientID, EventTime)
GROUP BY ClientID
LIMIT 5
FORMAT TabSeparatedWithNames
'''

get_clickhouse_df(q)

,ClientID,times,urls,step1_time,step2_time
0,1464279176664798753,"[1467651234,1467651273,1467651336,1467651425,1467651443,1467654366,1467747511,1467747533]","['supermarket.ru/main','supermarket.ru/catalog','supermarket.ru/product','supermarket.ru/finish_reserve','supermarket.ru/catalog','supermarket.ru/product','supermarket.ru/product','supermarket.ru/product']",1467651234,1467651273
1,146446839869868089,"[1467736293,1467736447,1467736479,1467736537]","['supermarket.ru/catalog','supermarket.ru/catalog','supermarket.ru/product','supermarket.ru/catalog']",0,0
2,1469184715541189697,"[1469195519,1469195578,1469195621,1469195654,1469195711,1469195745,1469195850]","['supermarket.ru/main','supermarket.ru/main','supermarket.ru/promo_action_1','supermarket.ru/main','supermarket.ru/search','supermarket.ru/product','supermarket.ru/product']",1469195519,0
3,1468981110571788812,[1468995858],['supermarket.ru/catalog'],0,0
4,1469025223567819473,"[1469036023,1469036035,1469036063,1469036094,1469036120,1469036136,1469036148,1469036190,1469036515,1469036660,1469036798,1469036815,1469036827,1469037142,1469037147,1469037168]","['supermarket.ru/main','supermarket.ru/search','supermarket.ru/product','supermarket.ru/search','supermarket.ru/product','supermarket.ru/product','supermarket.ru/product','supermarket.ru/product','supermarket.ru/product','supermarket.ru/product','supermarket.ru/promo','supermarket.ru/search','supermarket.ru/product','supermarket.ru/catalog','supermarket.ru/catalog','supermarket.ru/product']",1469036023,1469037142


In [32]:
q = '''
SELECT
    5*countIf(step1_time != 0) as step1,
    5*countIf(step2_time != 0) as step2,
    5*countIf(step3_time != 0) as step3,
    5*countIf(step4_time != 0) as step4,
    5*countIf(step5_time != 0) as step5,
    5*countIf(step6_time != 0) as step6
FROM
    (SELECT
        ClientID,
        groupArray(cast(EventTime as UInt64)) as times,
        groupArray(URL) as urls,
        arrayFilter(time, url -> url = 'supermarket.ru/main', times, urls)[1] as step1_time,
        arrayFilter(time, url -> url = 'supermarket.ru/catalog' and step1_time != 0 and time >= step1_time, times, urls)[1] as step2_time,
        arrayFilter(time, url -> url = 'supermarket.ru/product' and step2_time != 0 and time >= step2_time, times, urls)[1] as step3_time,
        arrayFilter(time, url -> url = 'supermarket.ru/basket' and step3_time != 0 and time >= step3_time, times, urls)[1] as step4_time,
        arrayFilter(time, url -> url = 'supermarket.ru/finish_order' and step4_time != 0 and time >= step4_time, times, urls)[1] as step5_time,
        arrayFilter(time, url -> url = 'supermarket.ru/user_profile' and step5_time != 0 and time >= step5_time, times, urls)[1] as step6_time
    FROM
        (SELECT * FROM visits SAMPLE 1/5 ORDER BY ClientID, EventTime)
    GROUP BY ClientID)
FORMAT TabSeparatedWithNames
'''

fdf = get_clickhouse_df(q)

In [33]:
phases = ['Step %d' % i for i in range(1, 6)]
values = fdf.iloc[0].values

plot_funnel(phases, values)

[500, 315, 212, 10, 2, 0]
